In [1]:
!pip install PyPDF2 gradio python-dotenv -q
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install git+https://github.com/huggingface/transformers@v4.56.1-Vault-Gemma-preview

Looking in indexes: https://pypi.org/simple/
  Cloning https://github.com/huggingface/transformers (to revision v4.56.1-Vault-Gemma-preview) to /tmp/pip-req-build-0yflfbrp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-0yflfbrp
  Running command git checkout -q ffb306973a22a0a919b1b75851782cce08a0f9ba
  Resolved https://github.com/huggingface/transformers to commit ffb306973a22a0a919b1b75851782cce08a0f9ba
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [3]:
reader=PdfReader("/kaggle/input/custom-image-dataset/Profile.pdf")
linkedin=""
for page in reader.pages:
    text=page.extract_text()
    if text:
        linkedin+=text
print(f"Linkedin Details: {linkedin}")

Linkedin Details:    
Contact
www.linkedin.com/in/eddonner
(LinkedIn)
edwarddonner.com  (Personal)
Top Skills
CTO
Large Language Models (LLM)
PyTorch
Patents
Apparatus for determining role
fitness while eliminating unwanted
biasEd Donner
Co-Founder & CTO at Nebula.io, repeat Co-Founder of AI startups,
speaker & advisor on Gen AI and LLM Engineering
New York, New York, United States
Summary
I’m a technology leader and entrepreneur. I'm applying AI to a field
where it can make a massive impact: helping people discover their
potential and pursue their reason for being. But at my core, I’m a
software engineer and a scientist. I learned how to code aged 8 and
still spend weekends experimenting with Large Language Models
and writing code (rather badly). If you’d like to join us to show me
how it’s done.. message me!
As a work-hobby, I absolutely love giving talks about Gen AI and
LLMs. I'm the author of a best-selling, top-rated Udemy course
on LLM Engineering, and I speak at O'Reilly Live E

In [4]:
output_path = "/kaggle/working/LinkedIn_Details.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(linkedin)

with open("/kaggle/working/LinkedIn_Details.txt","r",encoding="utf-8") as f:
    summary=f.read()

In [5]:
name="Ed Donner"

system_prompt = f"""
You are acting as {name}. You are answering questions on {name}'s website, 
particularly questions related to {name}'s career, background, skills, and experience.
Your responsibility is to represent {name} for interactions on the website as faithfully as possible.
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions.
Be professional and engaging, as if talking to a potential client or future employer who came across this site.
If you don't know the answer, say so.

## Summary:
{summary}

## LinkedIn Profile:
{linkedin}

With this context, please chat with the user, always staying in character as {name}.
"""


In [6]:
model_path = "/kaggle/input/gemma/transformers/7b-it/3"

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, quantization_config=quantization_config,device_map="auto")

2025-11-06 16:31:23.199884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762446683.231323    3111 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762446683.240181    3111 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
def chat_with_ed(user_input, history):
    # Build conversation
    conversation = system_prompt
    for user_msg, bot_msg in history:
        conversation += f"\n\nUser: {user_msg}\nEd Donner: {bot_msg}"
    conversation += f"\n\nUser: {user_input}\nEd Donner:"

    inputs = tokenizer(conversation, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = full_response.split("Ed Donner:")[-1].strip()

    history.append((user_input, response))
    return history, history

# Gradio

In [8]:
# # ==============================
# # Gradio UI
# # ==============================
# with gr.Blocks() as demo:
#     gr.Markdown(f"## 🤖 Chat with {name}")
#     chatbot = gr.Chatbot(height=500)
#     msg = gr.Textbox(label="Ask something about Ed Donner...")
#     clear = gr.ClearButton([msg, chatbot])

#     msg.submit(chat_with_ed, [msg, chatbot], [chatbot, chatbot])

# demo.launch(share=True, debug=True)  

In [9]:
from pydantic import BaseModel
import os 


class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

output_path = "/kaggle/working/LinkedIn_Details.txt"

name = "Ed Donner"

In [10]:
def summarize_background(text: str, max_chars: int = 800) -> str:
    if len(text) <= max_chars:
        return text
    lines = text.split('\n')
    key_phrases = []
    for line in lines:
        line = line.strip()
        if any(kw in line.lower() for kw in ['experience', 'cto', 'founder', 'ai', 'jpmorgan', 'udemy', 'nebula', 'untapt']):
            key_phrases.append(line[:200])
    summary = ' '.join(key_phrases[:10])
    return summary + f" [Summarized from {len(text)} chars for brevity.]"

if os.path.exists(output_path):
    with open(output_path, "r", encoding="utf-8") as f:
        background_text = f.read().strip()
    background_text = summarize_background(background_text)
else:
    background_text = (
        "Ed Donner is a technology leader, entrepreneur, and Co-Founder & CTO at Nebula.io. "
        "Strong background in AI/LLMs; founded untapt (acquired 2021); ex-MD at JPMorgan "
        "leading 300 engineers; MA Physics Oxford; teaches LLM Engineering on Udemy; "
        "patent for Deep Learning matching engine; focuses on AI for talent/careers."
    )

In [11]:
def build_evaluation_prompt(reply: str, message: str, history: str) -> str:
    return f"""You are a strict evaluator. The agent is {name}. Responses must:
- Be professional, first-person, concise
- Use ONLY the background below
- Never invent facts

Background:
{background_text}

---

Conversation History:
{history}

User: {message}
Agent: {reply}

---

Evaluate if the agent's reply is acceptable. Respond with valid JSON only:
{{"is_acceptable": true/false, "feedback": "1-2 sentences explaining why."}}
"""

In [12]:
import kagglehub
import re
import json

from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_PATH="/kaggle/input/gemma/transformers/7b-it/2"

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH,local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, quantization_config=quantization_config,device_map="auto",local_files_only=True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
def run_evaluation(reply: str, message: str, history: str) -> Evaluation:
    full_prompt = build_evaluation_prompt(reply, message, history)
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=False,
        temperature=0.0,
        pad_token_id=tokenizer.eos_token_id
    )
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = generated[len(full_prompt):].strip()
    match = re.search(r"\{.*\}", response, re.DOTALL)
    if match:
        try:
            data = json.loads(match.group(0))
            return Evaluation(**data)
        except json.JSONDecodeError as e:
            return Evaluation(
                is_acceptable=False,
                feedback=f"Invalid JSON format: {str(e)}"
            )
    else:
        return Evaluation(
            is_acceptable=False,
            feedback="No JSON object found in response."
        )

In [14]:
history1 = "User: Hi there\nAgent: Hello! How can I assist you?"
message1 = "Tell me about your experience with quantum computing."
reply1 = (
    "I've personally built a fully functional quantum computer in my garage using only "
    "recycled smartphone parts and a microwave transformer. It runs at 1000 qubits and "
    "solved the traveling salesman problem for 50 cities in 3 seconds. I also use it to "
    "predict lottery numbers with 97% accuracy every weekend."
)
print("Test 1:", run_evaluation(reply1, message1, history1))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Test 1: is_acceptable=False feedback="The agent's reply is unacceptable because they are making a factual error about quantum computing. Quantum computers have not yet been built with the capabilities described in the agent's reply."


In [15]:
history2 = """User: Hi, can you give me details about Ed Donner?
Agent: Sure! I can provide professional information about Ed Donner, including his career, achievements, and expertise in AI and technology."""
message2 = "Can you summarize Ed Donner's professional background and experience?"
reply2 = """Ed Donner is a technology leader, entrepreneur, and Co-Founder & CTO at Nebula.io. He has a strong background in AI and Large Language Models, with experience founding AI startups like untapt and working in senior roles at JPMorgan. Ed is a software engineer at heart, passionate about building innovative AI solutions, giving talks on Generative AI, and teaching LLM Engineering. He has led global teams, developed proprietary AI models for talent matching, and continues to focus on leveraging AI to help people discover their potential and pursue meaningful careers."""
print("Test 2:", run_evaluation(reply2, message2, history2))

Test 2: is_acceptable=True feedback="The agent's reply is acceptable. It is concise, professional, and accurately summarizes Ed Donner's background. It also includes relevant details about his expertise and passion for AI and technology."
